In [17]:
import json
from datetime import datetime

class Experience:
    def __init__(self, company, title, start, end):
        self.company = company
        self.title = title
        self.start = start
        self.end = end

class Person:
    def __init__(self, id, first, last, phone, experience):
        self.id = id
        self.first = first
        self.last = last
        self.phone = phone
        self.experience = experience

def load_persons_from_json(file_path):
    persons = dict()
    with open(file_path, 'r') as file:
        data = json.load(file)
        for item in data:
            id = item.get('id')
            first = item.get('first')
            last = item.get('last')
            phone = item.get('phone')
            if phone is not None and '-' not in phone:
                phone = phone[:1] + '-' + phone[1:5] + '-' + phone[5:]  # Add dash if missing
            experience_list = []
            for exp in item.get('experience', []):
                company = exp.get('company')
                title = exp.get('title')
                start = datetime.strptime(exp.get('start'), "%Y-%m-%d").date()
                end = datetime.strptime(exp.get('end'), "%Y-%m-%d").date() if exp.get('end') else None
                experience = Experience(company, title, start, end)
                experience_list.append(experience)
            person = Person(id, first, last, phone, experience_list)
            persons[person.id] = (person)
    return persons

In [18]:
file_path = 'persons.json'
persons = load_persons_from_json(file_path)
for person_id in persons:
    person = persons[person_id]
    print(person.id, person.first, person.last, person.phone, person.experience[0].company)


0 Jane Doe 1-2123458974 OrangeCart
1 Steve Jobs (212) 345-8974 Allari
2 Bill Gates 1-2123411113 OrangeCart
3 Tim Cook (-1) 2-12345897 Google
4 Alan Turing 5-9895-55120 Google
5 Bugs Bunny 1-2123458004 Pixar


In [19]:
def get_connected_persons_by_job(persons, person_id):
    connected = []

    if(person_id not in persons):
        print('Error: Person with ID not found')
        return

    target_person = persons[person_id]

    for key in persons:
        if(key != person_id):
            person = persons[key]
            if(person.experience[0].company == target_person.experience[0].company):
                connected.append(person)
        

    return connected

In [20]:
file_path = 'persons.json'
persons = load_persons_from_json(file_path)
coworkers = get_connected_persons(persons, 0)
print(coworkers[0].id)

file_path = 'persons.json'
persons = load_persons_from_json(file_path)
coworkers = get_connected_persons(persons, 2)
print(len(coworkers))

2
1


In [21]:
def overlaps_90_days(start1, end1, start2, end2):
    today = datetime.today().date()
    end_or_today1 = end1 or today
    end_or_today2 = end2 or today
         
    if(start1 <= end_or_today2 and end_or_today1 >= start2) or (start2 <= end_or_today1 and end_or_today2 >= start1):
        diff = 0
        # When one range contains another
        if(start1 <= start2) and (end_or_today1 >= end_or_today2):
            diff = end_or_today2 - start2
             
        # When one range is included in the other
        if(start2 <= start1) and (end_or_today1 <= end_or_today2):
            diff = end_or_today1 - start1

        # When ranges partially intercept - right
        if(start1 >= start2) and (end_or_today1 >= end_or_today2):
            diff = end_or_today2 - start1

        #When ranges partially intercept - left
        if(start1 <= start2) and (end_or_today1 <= end_or_today2):
            diff = end_or_today1 - start2
        
        return diff.days > 90
    else:
        return False
       


In [26]:
def have_worked_together(first_person, second_person):
    for exp1 in first_person.experience:
        for exp2 in second_person.experience:
            if exp1.company == exp2.company and overlaps_90_days(exp1.start, exp1.end, exp2.start, exp2.end):
                return True
    return False

In [28]:
def get_connected_persons_by_job(persons, person_id):
    connectedIds = []
    
    if(person_id not in persons):
        print('Error: Person with ID not found')
        return

    target_person = persons[person_id]
    
    for key in persons:
        if(key != person_id):
          person = persons[key]
          if(target_person != None) and have_worked_together(person, target_person):
            connectedIds.append(person.id)

    return connectedIds


print(get_connected_persons_by_job(persons,0))
print(get_connected_persons_by_job(persons,1))
print(get_connected_persons_by_job(persons,2))
print(get_connected_persons_by_job(persons,3))
print(get_connected_persons_by_job(persons,4))
print(get_connected_persons_by_job(persons,5))


[2]
[]
[0]
[4]
[3]
[]
